In [94]:
import torch
import torch.nn as nn

### <u>Compare functions one by one</u>

#### 1. MaxPool1d

In [95]:
#Create some random input
random_tensor = torch.rand(3, 4)
random_tensor

tensor([[0.6735, 0.5949, 0.8780, 0.8296],
        [0.3949, 0.9947, 0.2762, 0.1802],
        [0.7275, 0.5064, 0.8903, 0.1068]])

In [96]:
#define the pytorch equivalent
pytorch_max_pool_fn = nn.MaxPool1d(kernel_size=2, stride=1, padding=0,dilation=1, return_indices=False, ceil_mode=False)
output_from_pytorch = pytorch_max_pool_fn(random_tensor)
output_from_pytorch

tensor([[0.6735, 0.8780, 0.8780],
        [0.9947, 0.9947, 0.2762],
        [0.7275, 0.8903, 0.8903]])

In [97]:
#define my custom implementation of maxPool1d ( assumes that x is 2D (C,W) shape)
def my_MaxPool1d(x, kernel_size=2, stride=1, padding=0, dilation=1):
    result = []
    for row in x:
        maxPoolForRow = []
        #k represents start point of the kernel
        for k in range(0, len(row) - kernel_size + 1, stride):
            maxPoolForRow.append(max(row[k:k + kernel_size]))
        result.append(maxPoolForRow)
    result = torch.tensor(result)
    return result


output_from_my_implementation = my_MaxPool1d(random_tensor)
output_from_my_implementation

tensor([[0.6735, 0.8780, 0.8780],
        [0.9947, 0.9947, 0.2762],
        [0.7275, 0.8903, 0.8903]])

In [98]:
#Assert that both are the same
torch.equal(output_from_pytorch, output_from_my_implementation)

True

#### 2. AvgPool1d

In [99]:
#Create some random input
random_tensor = torch.rand(3, 4)
random_tensor

tensor([[0.5099, 0.7110, 0.6018, 0.9501],
        [0.6111, 0.8375, 0.8109, 0.6189],
        [0.7721, 0.2097, 0.9601, 0.4213]])

In [100]:
#define the pytorch equivalent
pytorch_avg_pool_fn = nn.AvgPool1d(kernel_size=2, stride=1, padding=0,ceil_mode=False, count_include_pad=True)
output_from_pytorch = pytorch_avg_pool_fn(random_tensor)
output_from_pytorch


tensor([[0.6104, 0.6564, 0.7760],
        [0.7243, 0.8242, 0.7149],
        [0.4909, 0.5849, 0.6907]])

In [101]:
#define my custom implementation of avgPool1d ( assumes that x is 2D (C,W) shape)
def my_AvgPool1d(x, kernel_size=2, stride=1, padding=0, dilation=1):
    result = []
    for row in x:
        maxPoolForRow = []
        #k represents start point of the kernel
        for k in range(0, len(row) - kernel_size + 1, stride):
            maxPoolForRow.append(sum(row[k:k + kernel_size])/kernel_size)
        result.append(maxPoolForRow)
    result = torch.tensor(result)
    return result


output_from_my_implementation = my_AvgPool1d(random_tensor)
output_from_my_implementation

tensor([[0.6104, 0.6564, 0.7760],
        [0.7243, 0.8242, 0.7149],
        [0.4909, 0.5849, 0.6907]])

In [102]:
#Assert that both are the same
torch.equal(output_from_pytorch, output_from_my_implementation)

True

#### 3. Conv1d

In [103]:
#Loading the given filter and input tensors
pixel_input = torch.load("./pixel_input.pt") #Shape (1,1,32)
filter = torch.load("./filter.pt") #(Shape 3,1,1)

In [104]:
#define pytorch equivalent
pytorch_conv1d_output = nn.functional.conv1d(pixel_input, filter, bias = None,stride = 1, padding = 0, dilation = 1, groups = 1)

#Shape is (1,3,32)

In [105]:
#define custom implementation
def my_Conv1d(inp, filter, stride=1, padding=0, dilation=1, groups = 1):
    result = []

    for f in filter:
        #as f in this case is 1x1 ( but it could in theory be something like 1 x k)
        f = f[0]
        batch_row = []
        for batch in inp: #(1,32)
            conv_row = []
            for channel in batch: #(32,)
                #convolve f over this channel array using this filter by using dot product to convolve!
                for f_ind in range(0, len(channel) - len(f) + 1, stride):
                    conv_row.append(torch.dot(f, channel[f_ind: f_ind + len(f)]))
            batch_row.append(conv_row)
        result.append(batch_row)

    result = torch.tensor(result)

    #permute it so that the shapes are correct.
    result = result.permute(1,0,2)
    return result

my_conv1d_output = my_Conv1d(pixel_input, filter)

In [106]:
#Assert that both are the same
torch.equal(pytorch_conv1d_output, my_conv1d_output)

True

#### 4. Sigmoid



In [107]:
#Create some random input
random_tensor = torch.rand(2, 2)
random_tensor

tensor([[0.5459, 0.0456],
        [0.0720, 0.1420]])

In [108]:
#define the pytorch equivalent
pytorch_sigmoid_fn = nn.Sigmoid()
output_from_pytorch = pytorch_sigmoid_fn(random_tensor)
output_from_pytorch


tensor([[0.6332, 0.5114],
        [0.5180, 0.5354]])

In [109]:
#define my custom implementation of maxPool1d ( assumes that x is 2D (C,W) shape)
def my_Sigmoid(x):
    return 1/(1+torch.exp(-x))

output_from_my_implementation = my_Sigmoid(random_tensor)
output_from_my_implementation

tensor([[0.6332, 0.5114],
        [0.5180, 0.5354]])

In [110]:
#Assert that both are the same
torch.equal(pytorch_conv1d_output, my_conv1d_output)

True

#### 5. BatchNorm1d

In [111]:
#Create some random input
random_tensor = torch.rand(2,1,4) #(batchsize,numfeatures, actualfeatures)
random_tensor

tensor([[[0.1715, 0.7690, 0.0808, 0.1541]],

        [[0.3075, 0.4730, 0.8076, 0.8334]]])

In [112]:
#define the pytorch equivalent
pytorch_batchnorm_fn = nn.BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)
output_from_pytorch = pytorch_batchnorm_fn(random_tensor)
output_from_pytorch = output_from_pytorch.data.detach()
output_from_pytorch

tensor([[[-0.9408,  1.0804, -1.2479, -0.9997]],

        [[-0.4807,  0.0791,  1.2110,  1.2985]]])

In [113]:
#define my custom implementation of batchnorm ( assumes that x is 3D (B,1,N) shape)
def my_BatchNorm1d(x, eps=1e-05, momentum=0.1, affine=True):
    #momentum is used only during training and not used during inference!
    # Compute batch mean and batch variance
    batch_mean = x.mean(dim=(0, 2), keepdim=True)
    batch_var = x.var(dim=(0, 2), unbiased=False, keepdim=True)

    # Normalize the input using batch mean and variance
    x_normalized = (x - batch_mean) / torch.sqrt(batch_var + eps)

    return x_normalized

output_from_my_implementation = my_BatchNorm1d(random_tensor)
output_from_my_implementation

tensor([[[-0.9408,  1.0804, -1.2479, -0.9997]],

        [[-0.4807,  0.0791,  1.2110,  1.2985]]])

In [114]:
#Assert that both are the same
torch.allclose(output_from_pytorch, output_from_my_implementation, rtol=1e-05, atol=1e-05)

True

#### 6. Linear

In [115]:
#Create some random input
random_tensor = torch.rand(4,3)
random_tensor

tensor([[0.2100, 0.7635, 0.4117],
        [0.7382, 0.9847, 0.2794],
        [0.4901, 0.1510, 0.3108],
        [0.9837, 0.4750, 0.9455]])

In [116]:
#define the pytorch equivalent
in_features = 3
out_features = 5
pytorch_linear_fn = nn.Linear(in_features, out_features, bias=True)
output_from_pytorch = pytorch_linear_fn(random_tensor)
output_from_pytorch = output_from_pytorch.data.detach()
output_from_pytorch

tensor([[-0.4839,  0.8569,  0.2203,  0.3922, -0.9922],
        [-0.3001,  0.9489,  0.2005,  0.4876, -0.8936],
        [-0.0809,  0.6080, -0.1912,  0.0976, -0.5068],
        [ 0.1376,  1.0333, -0.2396, -0.1431, -0.9159]])

In [117]:
#define my custom implementation of batchnorm ( assumes that x is 3D (B,1,N) shape)
def my_Linear(x, in_features=3, out_features=5):
    weights = torch.randn(in_features, out_features)  # Corrected order
    biases = torch.randn(out_features)

    # Perform linear transformation
    return torch.matmul(x, weights) + biases

output_from_my_implementation = my_Linear(random_tensor)
output_from_my_implementation

tensor([[ 0.9288,  2.0881,  0.9052, -0.6827, -2.0643],
        [ 1.0260,  2.5993,  0.7594, -1.2396, -2.8089],
        [ 1.7913,  1.9512,  0.6871, -0.3314, -0.8802],
        [ 2.7243,  2.7840,  1.6021, -1.1745, -0.5432]])

In [118]:
#Assert that both are the same size as the data inside may be random
output_from_pytorch.shape == output_from_my_implementation.shape

True